<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get profile from user
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+profile+from+user:+Error+short+description">Bug report</a>

**Tags:** #github #user #profile #operations #snippet #dataframe

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-25 (Created: 2022-03-18)

**Description:** This notebook provides a way to retrieve a user's profile information from GitHub.

## Input

### Import libraries

In [1]:
from naas_drivers import github
import naas
import requests
import pandas as pd

### Setup variables

- `github_username`: user profile
- `GITHUB_TOKEN`: token to access github information
- `output_csv`: excel file

In [3]:
github_username = "FlorentLvr"
GITHUB_TOKEN = naas.secret.get("GITHUB_TOKEN")
output_csv = f"{github_username}_profile.csv"

## Model

### Get profile from user

In [4]:
def get_github_user_info(username, token=None):
    url = f"https://api.github.com/users/{username}"
    headers = {"Authorization": f"token {token}"} if token else {}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_commit_emails(username, token=None):
    url = f"https://api.github.com/users/{username}/events/public"
    headers = {"Authorization": f"token {token}"} if token else {}
    response = requests.get(url, headers=headers)
    emails = set()
    
    if response.status_code == 200:
        events = response.json()
        for event in events:
            if event['type'] == 'PushEvent':
                for commit in event['payload']['commits']:
                    emails.add(commit['author']['email'])
        return emails
    else:
        return f"Error: {response.status_code} - {response.json().get('message', 'Unknown error')}"


# Fetch basic GitHub user information
github_user_info = get_github_user_info(github_username, GITHUB_TOKEN)

if github_user_info:
    email = github_user_info.get('email')
    if email:
        print(f"Email: {email}")
    else:
        print("Email not found in GitHub profile. Checking commit history...")
        # If no email is found in the profile, check commit history
        emails = get_commit_emails(github_username, GITHUB_TOKEN)
        if isinstance(emails, set):
            if emails:
                print(f"Found emails: {emails}")
            else:
                print("No emails found in commit history.")
        else:
            print(emails)
else:
    print("GitHub user not found.")
    
github_user_info

## Output

### Display result

In [5]:
data = []

data.append({
    "LOGIN": github_user_info["login"],
    "ID": github_user_info["id"],
    "NODE_ID": github_user_info["node_id"],
    "TYPE": github_user_info["type"],
    "NAME": github_user_info["name"],
    "COMPANY": github_user_info["company"],
    "LOCATION": github_user_info["location"],
    "EMAILS_IF_FOUND": emails,
    "HIREABLE": github_user_info["hireable"],
    "BIO": github_user_info["bio"],
    "TWITTER_USERNAME": github_user_info["twitter_username"],
    "PUBLIC_REPOS": github_user_info["public_repos"],
    "PUBLIC_GISTS": github_user_info["public_gists"],
    "FOLLOWERS": github_user_info["followers"],
    "FOLLOWING": github_user_info["following"],
    "CREATED_AT": github_user_info["created_at"],
    "UPDATED_AT": github_user_info["updated_at"],
})

df = pd.DataFrame(data)
df

### Save dataframe to csv

In [7]:
df.to_csv(output_csv, index=False)